# Unit conversion notes

## Follow-up work

 - Change name to Display Units
 - Which plugins will break
   - Cubeviz slider
   - Model fitting
   - Line analysis
 - Floating point conversion error?
 - Mouse events account for display units
 - Default to first data loaded for display units
 - Global units which plugins can listen to. Maybe a display_unit_changed message?
 - Should get data listen to display units?
 - Feature branch or develop in place?
---

## Acceptance criteria

 - Equivalencies list in notebook
 - Make tickets
 ---
 
## Examples 
 Can make multiple unit_converter instances, which can handle different cases in Jdaviz
 
 - from glue.config import settings
 - settings.UNIT_CONVERTER = 'custom-astropy'
 - @unit_converter('custom-jdaviz')
 - class UnitConverterWithSpectral:

Using d1 below:
```
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['FREQ']
wcs1.wcs.crval = [1]
wcs1.wcs.cdelt = [1]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['GHz']

d1 = Data(f1=[1, 2, 3])
d1.get_component('f1').units = 'Jy'
d1.coords = wcs1
```
access with
```
astropy_converter = unit_converter.members[settings.UNIT_CONVERTER]()
equivalent_units = astropy_converter.equivalent_units(d1, d1.components[0], 'mJy')
converted_units = astropy_converter.to_unit(d1, 'Pixel Axis 0 [x]', d1.x, 'GHz', 'centimeter')
```
---
Dev notes

y display unit is None and x display unit is set to 'Hz'
y axis label is Data values
x label is Frequency \[Hz\]

In [ ]:
import warnings

import numpy as np
from astropy.wcs import WCS
import glue_jupyter as gj
from glue.core import Data
from glue.plugins.wcs_autolinking.wcs_autolinking import WCSLink
from specutils import Spectrum1D
from glue.core.link_helpers import LinkSame
from glue.core.roi import XRangeROI
from glue.config import settings


from jdaviz import Specviz

In [ ]:
def units_to_strings(unit_list):
    """Convert equivalencies into readable versions of the units.

    Parameters
    ----------
    unit_list : list
        List of either `astropy.units` or strings that can be converted
        to `astropy.units`.

    Returns
    -------
    result : list
        A list of the units with their best (i.e., most readable) string version.
    """
    all_units = []
    for unit in unit_list: 
        try:
            u.Unit(unit)
        except TypeError as te:
            # print(f"{unit}: {te}")
            # warnings.warn(f"{unit}: {te}")
            continue
        if u.Unit(unit) == u.Unit("Angstrom"):
            all_units.append(u.Unit(unit.name))
        elif hasattr(u.Unit(unit), "long_names") and len(u.Unit(unit).long_names) > 0:
            all_units.append(u.Unit(unit).long_names[0])
        else:
            all_units.append(u.Unit(unit).to_string())
            
    return all_units
        

    # return [u.Unit(unit).name
    #         if u.Unit(unit) == u.Unit("Angstrom")
    #         else u.Unit(unit).long_names[0] if (
    #             hasattr(u.Unit(unit), "long_names") and len(u.Unit(unit).long_names) > 0)
    #         else u.Unit(unit).to_string()
    #         for unit in unit_list]

In [ ]:
from astropy import units as u
from glue.core.units import unit_converter

@unit_converter('custom-jdaviz')
class UnitConverterWithSpectral:
    
    def create_flux_equivalencies_list(self, data, cid, units):
        spec = data.get_object(cls=Spectrum1D)
        equivalencies = u.spectral_density(np.sum(spec.spectral_axis))
        # Get local units.
        locally_defined_flux_units = ['Jy', 'mJy', 'uJy',
                                      'W / (m2 Hz)',
                                      'eV / (s m2 Hz)',
                                      'erg / (s cm2)',
                                      'erg / (s cm2 um)',
                                      'erg / (s cm2 Angstrom)',
                                      'erg / (s cm2 Hz)',
                                      'ph / (s cm2 um)',
                                      'ph / (s cm2 Angstrom)',
                                      'ph / (s cm2 Hz)']
        local_units = [u.Unit(unit) for unit in locally_defined_flux_units]

        # Remove overlap units.
        curr_flux_unit_equivalencies = u.Unit(
            spec.flux.unit).find_equivalent_units(
                equivalencies=equivalencies,
                include_prefix_units=False)

        curr_flux_unit_equivalencies = list(set(curr_flux_unit_equivalencies) - set(local_units))
        flux_unit_equivalencies_titles = sorted(units_to_strings(curr_flux_unit_equivalencies)) + sorted(units_to_strings(local_units))

        return flux_unit_equivalencies_titles
    
    def create_spectral_equivalencies_list(self, spectrum,
                                       exclude=[u.jupiterRad, u.earthRad, u.solRad,
                                                u.lyr, u.AU, u.pc]):
        """
        Get all possible conversions from current spectral_axis_unit.
        """
        if spectrum.spectral_axis.unit == u.pix:
            return []

        # Get unit equivalencies.
        curr_spectral_axis_unit_equivalencies = u.Unit(
            spectrum.spectral_axis.unit).find_equivalent_units(
            equivalencies=u.spectral())

        # Get local units.
        locally_defined_spectral_axis_units = ['angstrom', 'nanometer',
                                               'micron', 'hertz', 'erg']
        local_units = [u.Unit(unit) for unit in locally_defined_spectral_axis_units]

        # Remove overlap units.
        curr_spectral_axis_unit_equivalencies = list(set(curr_spectral_axis_unit_equivalencies)
                                                     - set(local_units+exclude))

        # Convert equivalencies into readable versions of the units and sorted alphabetically.
        spectral_axis_unit_equivalencies_titles = sorted(units_to_strings(
            curr_spectral_axis_unit_equivalencies))

        # Concatenate both lists with the local units coming first.
        spectral_axis_unit_equivalencies_titles = units_to_strings(
            local_units) + spectral_axis_unit_equivalencies_titles

        return spectral_axis_unit_equivalencies_titles

    def equivalent_units(self, data, cid, units):
        spec = data.get_object(cls=Spectrum1D)
        
        # Here we will check if the cid is a flux axis or spectral axis.
        # We then either retrieve the list of spectral equivalencies or
        # flux equivalencies.
        print("equivalent_units", cid)
        if cid in ['f1', 'f2']:
            equivalencies = self.create_flux_equivalencies_list(data, cid, units)
        else:
            equivalencies = self.create_spectral_equivalencies_list(spec)
        return map(str, equivalencies)


    def to_unit(self, data, cid, values, original_units, target_units):
        equivalencies = self.equivalent_units(data, cid, target_units)
        
        # Something like this for flux equivalencies
        print("to unit", cid)
        if cid in ['f1', 'f2']:
            return (values * u.Unit(original_units)).to_value(u.Unit(target_units), equivalencies=equivalencies)
        else:
            return values.to(target_units)

In [ ]:
settings.UNIT_CONVERTER = 'custom-jdaviz'

# The following cell changes the display of the flux unit

In [ ]:
wcs1 = WCS(naxis=1)
wcs1.wcs.ctype = ['FREQ']
wcs1.wcs.crval = [1]
wcs1.wcs.cdelt = [1]
wcs1.wcs.crpix = [1]
wcs1.wcs.cunit = ['GHz']

d1 = Data(f1=[1, 2, 3])
d1.get_component('f1').units = 'Jy'
d1.coords = wcs1

wcs2 = WCS(naxis=1)
wcs2.wcs.ctype = ['WAVE']
wcs2.wcs.crval = [10]
wcs2.wcs.cdelt = [10]
wcs2.wcs.crpix = [1]
wcs2.wcs.cunit = ['cm']

d2 = Data(f2=[2000, 1000, 3000])
d2.get_component('f2').units = 'mJy'
d2.coords = wcs2

specviz = Specviz()

In [ ]:
specviz.app.add_data(d1, "d1")
specviz.app.add_data(d2, "d2")
specviz.app.data_collection.add_link(WCSLink(d1, d2))

specviz.show()

# Take a moment here to add data manually from the data drop down into the viewer.

In [ ]:
def convert_x_axis_to(new_x):
    specviz.app.get_viewer("spectrum-viewer").state.x_display_unit = new_x

def convert_y_axis_to(new_y):
    specviz.app.get_viewer("spectrum-viewer").state.y_display_unit = new_y

In [ ]:
convert_y_axis_to("Jy")

In [ ]:
# Errors out because of an issue with equivalencies

convert_x_axis_to("cm")

In [ ]:
astropy_converter = unit_converter.members[settings.UNIT_CONVERTER]()

data = specviz.app.data_collection[0]
spec = data.get_object(cls=Spectrum1D)
print(list(astropy_converter.equivalent_units(data, 'Pixel Axis 0 [x]', 'GHz')))
print(astropy_converter.to_unit(data, 'Pixel Axis 0 [x]', spec.spectral_axis, spec.spectral_axis.unit, 'centimeter'))

In [ ]:
# Errors out because of an issue with equivalencies

print(list(astropy_converter.equivalent_units(data, 'f1', 'Jy')))
print(astropy_converter.to_unit(data, 'f1', spec.flux, spec.flux.unit, 'Jy'))

In [ ]:
for x in specviz.app.data_collection:
    print(x.components)

In [ ]:
specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(1.2e+8, 2e+9))